In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from plotly import graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px

In [5]:
# Le fichier comporte une ligne d'en-tête supplémentaire qui réduit l'ouverture du fichier à une seule colonne après export
# Or nous voulons avoir accès à l'intégralité des données pour une première exploration
# Nous allons utiliser l'argument skiprows pour ne pas prendre en compte cet en-tête
df = pd.read_csv('NH.Ts+dSST.csv', sep = ',', skiprows=1)
df.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
0,1880,-0.35,-0.51,-.22,-.30,-.06,-.16,-.18,-.26,-.22,-.31,-.42,-.39,-.28,***,***,-.19,-.20,-.32
1,1881,-0.30,-0.22,-.03,.00,.03,-.34,.08,-.05,-.27,-.44,-.37,-.24,-.18,-.19,-.30,.00,-.10,-.36
2,1882,0.26,0.21,.01,-.32,-.25,-.31,-.28,-.15,-.24,-.52,-.32,-.67,-.22,-.18,.08,-.19,-.25,-.36
3,1883,-0.57,-0.66,-.15,-.29,-.24,-.13,-.04,-.22,-.32,-.15,-.42,-.14,-.28,-.32,-.64,-.23,-.13,-.29
4,1884,-0.16,-0.09,-.62,-.59,-.36,-.43,-.39,-.49,-.45,-.44,-.57,-.47,-.42,-.39,-.13,-.52,-.44,-.49


In [6]:
# On examine les informations disponibles pour observer les types de données et éventuelles valeurs manquantes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    145 non-null    int64  
 1   Jan     145 non-null    float64
 2   Feb     145 non-null    float64
 3   Mar     145 non-null    object 
 4   Apr     145 non-null    object 
 5   May     145 non-null    object 
 6   Jun     145 non-null    object 
 7   Jul     145 non-null    object 
 8   Aug     145 non-null    object 
 9   Sep     145 non-null    object 
 10  Oct     145 non-null    object 
 11  Nov     145 non-null    object 
 12  Dec     145 non-null    object 
 13  J-D     145 non-null    object 
 14  D-N     145 non-null    object 
 15  DJF     145 non-null    object 
 16  MAM     145 non-null    object 
 17  JJA     145 non-null    object 
 18  SON     145 non-null    object 
dtypes: float64(2), int64(1), object(16)
memory usage: 21.7+ KB


In [7]:
# On remarque dès les premières lignes du tableau que des valeurs sont manquantes mais pas sous codification N/A ou NaN
# Nous allons opérer une recherche sur toutes les colonnes de type object pour vérifier les occurences de '***'
# L'idée étant de récupérer le pourcentage de données impactées afin de décider quoi faire de ces singularités.

variables = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec','J-D', 'D-N', 'DJF', 'MAM', 'JJA', 'SON']
missing_data = df[variables].eq('***')
print(missing_data.mean()*100)

Jan    0.000000
Feb    0.000000
Mar    0.689655
Apr    0.689655
May    0.689655
Jun    0.689655
Jul    0.689655
Aug    0.689655
Sep    0.689655
Oct    0.689655
Nov    0.689655
Dec    0.689655
J-D    0.689655
D-N    1.379310
DJF    0.689655
MAM    0.689655
JJA    0.689655
SON    0.689655
dtype: float64


In [8]:
# L'ensemble de ces colonnes semblent impactées. Ce sont soit des colonnes présentant une température relevée mensuelle,
# soit des colonnes qui condensent en moyenne les résultats des mois de l'année étudiée. 
# Il existe très peu de données manquantes mais la manière choisie
# pour les remplacer pourrait avoir un impact sur les futurs calculs et rendus.Nous y reviendrons.

# Procédons d'abord au conversion aux bons formats pour chaque variable.

In [9]:
# On remarque que les deux premiers mois (Jan, Feb) sont au format float64 quand les 10 autres sont au format object
# Cela va nous poser des problèmes lors du traitement et l'analyse des données. Par ailleurs,
# ce format nous empêche de vérifier globalement si des valeurs sont effectivement manquantes.
# Il s'agit donc d'opérer une conversion de tous les objects mois vers un format float

months = ['Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
df[months] =df[months].apply(pd.to_numeric, errors='coerce')

# on revérifie que la conversion s'est bien opérée :
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    145 non-null    int64  
 1   Jan     145 non-null    float64
 2   Feb     145 non-null    float64
 3   Mar     144 non-null    float64
 4   Apr     144 non-null    float64
 5   May     144 non-null    float64
 6   Jun     144 non-null    float64
 7   Jul     144 non-null    float64
 8   Aug     144 non-null    float64
 9   Sep     144 non-null    float64
 10  Oct     144 non-null    float64
 11  Nov     144 non-null    float64
 12  Dec     144 non-null    float64
 13  J-D     145 non-null    object 
 14  D-N     145 non-null    object 
 15  DJF     145 non-null    object 
 16  MAM     145 non-null    object 
 17  JJA     145 non-null    object 
 18  SON     145 non-null    object 
dtypes: float64(12), int64(1), object(6)
memory usage: 21.7+ KB


In [10]:
# Il en est de même pour les dernières colonnes du tableau, qui sont des agrégats et devraient être de type numériques pour 
# maintenir la cohérence avec les colonnes présentant les températures mensuelles.

other_variables = ['J-D', 'D-N', 'DJF', 'MAM', 'JJA', 'SON']
df[other_variables] =df[other_variables].apply(pd.to_numeric, errors='coerce')

In [11]:
# on revérifie que la conversion s'est bien opérée :
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    145 non-null    int64  
 1   Jan     145 non-null    float64
 2   Feb     145 non-null    float64
 3   Mar     144 non-null    float64
 4   Apr     144 non-null    float64
 5   May     144 non-null    float64
 6   Jun     144 non-null    float64
 7   Jul     144 non-null    float64
 8   Aug     144 non-null    float64
 9   Sep     144 non-null    float64
 10  Oct     144 non-null    float64
 11  Nov     144 non-null    float64
 12  Dec     144 non-null    float64
 13  J-D     144 non-null    float64
 14  D-N     143 non-null    float64
 15  DJF     144 non-null    float64
 16  MAM     144 non-null    float64
 17  JJA     144 non-null    float64
 18  SON     144 non-null    float64
dtypes: float64(18), int64(1)
memory usage: 21.7 KB


In [12]:
df.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
0,1880,-0.35,-0.51,-0.22,-0.30,-0.06,-0.16,-0.18,-0.26,-0.22,-0.31,-0.42,-0.39,-0.28,NaN,NaN,-0.19,-0.20,-0.32
1,1881,-0.30,-0.22,-0.03,0.00,0.03,-0.34,0.08,-0.05,-0.27,-0.44,-0.37,-0.24,-0.18,-0.19,-0.30,0.00,-0.10,-0.36
2,1882,0.26,0.21,0.01,-0.32,-0.25,-0.31,-0.28,-0.15,-0.24,-0.52,-0.32,-0.67,-0.22,-0.18,0.08,-0.19,-0.25,-0.36
3,1883,-0.57,-0.66,-0.15,-0.29,-0.24,-0.13,-0.04,-0.22,-0.32,-0.15,-0.42,-0.14,-0.28,-0.32,-0.64,-0.23,-0.13,-0.29
4,1884,-0.16,-0.09,-0.62,-0.59,-0.36,-0.43,-0.39,-0.49,-0.45,-0.44,-0.57,-0.47,-0.42,-0.39,-0.13,-0.52,-0.44,-0.49


In [13]:
# Nous avons vu que certaines colonnes du tableau faisaient préalablement mention de données de type '***'
# L'opération de conversion au format float64 nous les présente désormais en tant que NaN. Il va s'agir de les remplacer.

# Quelle méthode choisir ? En effet des méthodes globales (ensemble de la colonne) ne semblent pas pertinentes car des données 
# actuelles avec des températures plus élevées pourraient polluer notre jeu de données lors de la conversion (agissant en 
# outliers). Possibilité cependant : réduire la méthode choisie à un échantillon 
# de la décennie dont font parties les données manquantes, exemple ici, nous choisirons la moyenne sur 10 ans.

# On définit des groupes de décennies
groupes_decennies = [(1880, 1890), (1890, 1900), (1900, 1910),
                     (1910, 1920), (1920, 1930), (1930, 1940),
                     (1940, 1950), (1950, 1960), (1960, 1970),
                     (1970, 1980), (1980, 1990), (1990, 2000),
                     (2000, 2010), (2010, 2020), (2020, 2024)]

# On filtre le dataframe df en fonction des groupes de décennies
df_groupes = pd.DataFrame()
for groupe in groupes_decennies:
    borne_inf, borne_sup = groupe
    df_filtre = df[(df['Year'] >= borne_inf) & (df['Year'] < borne_sup)]
    df_groupes = pd.concat([df_groupes, df_filtre]) 
    
# On sélectionne les colonnes comportant les valeurs '***' à remplacer
colonnes_a_remplacer = ['Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec','J-D', 'D-N', 'DJF', 'MAM', 'JJA', 'SON']

# On calcule la moyenne par décennie
moyennes_decennie = df_groupes.groupby(df_groupes['Year'] // 10 * 10)[colonnes_a_remplacer].mean()

# On crée une fonction pour remplacer les NaN par la moyenne par décennie
def remplacer_nan_par_moyenne(row):
    year = row['Year']
    for colonne in colonnes_a_remplacer:
        moyenne_decennie = moyennes_decennie.loc[year // 10 * 10, colonne]
        if pd.isna(row[colonne]):
            row[colonne] = round(moyenne_decennie, 2)
    return row

# Appliquer la fonction personnalisée à chaque ligne du DataFrame
df = df.apply(remplacer_nan_par_moyenne, axis=1)

df.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
0,1880.0,-0.35,-0.51,-0.22,-0.30,-0.06,-0.16,-0.18,-0.26,-0.22,-0.31,-0.42,-0.39,-0.28,-0.31,-0.38,-0.19,-0.20,-0.32
1,1881.0,-0.30,-0.22,-0.03,0.00,0.03,-0.34,0.08,-0.05,-0.27,-0.44,-0.37,-0.24,-0.18,-0.19,-0.30,0.00,-0.10,-0.36
2,1882.0,0.26,0.21,0.01,-0.32,-0.25,-0.31,-0.28,-0.15,-0.24,-0.52,-0.32,-0.67,-0.22,-0.18,0.08,-0.19,-0.25,-0.36
3,1883.0,-0.57,-0.66,-0.15,-0.29,-0.24,-0.13,-0.04,-0.22,-0.32,-0.15,-0.42,-0.14,-0.28,-0.32,-0.64,-0.23,-0.13,-0.29
4,1884.0,-0.16,-0.09,-0.62,-0.59,-0.36,-0.43,-0.39,-0.49,-0.45,-0.44,-0.57,-0.47,-0.42,-0.39,-0.13,-0.52,-0.44,-0.49


In [14]:
# On vérifie s'il existe encore des valeurs manquantes dans le jeu de données :
df.isna().sum()

Year    0
Jan     0
Feb     0
Mar     0
Apr     0
May     0
Jun     0
Jul     0
Aug     0
Sep     0
Oct     0
Nov     0
Dec     0
J-D     0
D-N     0
DJF     0
MAM     0
JJA     0
SON     0
dtype: int64

In [15]:
# Pour être tout à fait complet, Il conviendrait par souci de propreté de modifier le format de la colonne Year
# il s'agit en effet ici d'une variable catégorielle dont nous ne tirerons aucun avantage statistique.
# Cependant des dates antérieures à 1970 nous empêchent de réaliser cette action.
# Nous pouvons cependant repasser Year en int64 et non plus en float64 comme auparavant.

df['Year'] = df['Year'].astype(int)
df.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
0,1880,-0.35,-0.51,-0.22,-0.30,-0.06,-0.16,-0.18,-0.26,-0.22,-0.31,-0.42,-0.39,-0.28,-0.31,-0.38,-0.19,-0.20,-0.32
1,1881,-0.30,-0.22,-0.03,0.00,0.03,-0.34,0.08,-0.05,-0.27,-0.44,-0.37,-0.24,-0.18,-0.19,-0.30,0.00,-0.10,-0.36
2,1882,0.26,0.21,0.01,-0.32,-0.25,-0.31,-0.28,-0.15,-0.24,-0.52,-0.32,-0.67,-0.22,-0.18,0.08,-0.19,-0.25,-0.36
3,1883,-0.57,-0.66,-0.15,-0.29,-0.24,-0.13,-0.04,-0.22,-0.32,-0.15,-0.42,-0.14,-0.28,-0.32,-0.64,-0.23,-0.13,-0.29
4,1884,-0.16,-0.09,-0.62,-0.59,-0.36,-0.43,-0.39,-0.49,-0.45,-0.44,-0.57,-0.47,-0.42,-0.39,-0.13,-0.52,-0.44,-0.49


In [16]:
# On peut désormais examiner la distribution statistiques de chaque colonne de df, en faisant abstraction de la colonne Year
# qui comme nous l'avons vu n'est pas concernée.

description = df.iloc[:, 1:].describe()
display(description)

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
count,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000
mean,0.100276,0.115724,0.138069,0.106069,0.097931,0.093931,0.096276,0.083517,0.102345,0.153931,0.138621,0.080483,0.108276,0.106552,0.095172,0.114414,0.091448,0.131586
std,0.587491,0.594265,0.575940,0.489260,0.425730,0.423724,0.406202,0.436828,0.445459,0.484234,0.524462,0.541323,0.464762,0.463445,0.543473,0.484498,0.417822,0.473738
min,-1.500000,-0.950000,-0.780000,-0.650000,-0.730000,-0.520000,-0.580000,-0.770000,-0.800000,-0.840000,-0.820000,-1.120000,-0.570000,-0.570000,-1.040000,-0.710000,-0.530000,-0.720000
25%,-0.290000,-0.350000,-0.240000,-0.260000,-0.210000,-0.180000,-0.180000,-0.200000,-0.200000,-0.170000,-0.220000,-0.280000,-0.220000,-0.200000,-0.270000,-0.240000,-0.200000,-0.200000
50%,0.050000,-0.010000,0.010000,0.000000,0.010000,-0.020000,-0.010000,-0.020000,-0.020000,0.050000,0.050000,0.000000,0.000000,0.010000,-0.010000,0.000000,-0.020000,0.030000
75%,0.400000,0.500000,0.420000,0.320000,0.250000,0.240000,0.250000,0.250000,0.260000,0.330000,0.350000,0.370000,0.260000,0.270000,0.400000,0.300000,0.220000,0.280000
max,1.640000,1.950000,1.900000,1.470000,1.270000,1.200000,1.430000,1.570000,1.660000,1.890000,1.990000,1.860000,1.490000,1.430000,1.810000,1.500000,1.390000,1.850000


In [23]:
df['Jan'].describe()

count    145.000000
mean       0.100276
std        0.587491
min       -1.500000
25%       -0.290000
50%        0.050000
75%        0.400000
max        1.640000
Name: Jan, dtype: float64